# Analysis of Survey Data I: Data Wrangling

In here I will read in the raw survey data and prepare it for analysis. The raw survey data is not included in this repo, because Qualtrics by default collects potentially identifying information (IP, geolocation, etc.).

The cleaned survey data can be found [here](https://github.com/MWiechmann/ai_story_scale/tree/main/study1_scale_construction/data).

Data was collected from two sources:
* Community sample: Recruited from community forums of users of AI storytelling apps. I asked people on the NovelAI discord, [NovelAI discord](https://discord.com/invite/novelai), [NovelAI Reddit](https://discord.com/invite/novelai) and the [AI Multiverse discord](https://discord.com/invite/puRyrw869h) for their participation.
* Panel sample: Recruited from panels for academic research ([SurveySwap.io](https://surveyswap.io/) and [SurveyCircle.com](https://www.surveycircle.com/)).

## Read In Data To Dataframes

In [1]:
import pandas as pd
import textstat as txt

# Response data from community and panel sample
community_df = pd.read_csv("data/raw/story_scale_community.csv")
panel_df = pd.read_csv(
    "data/raw/story_scale_panel.csv")

# Data from story extracts used for survey
story_df = pd.read_csv(
    "survey/NAI_story_data/NAI_story_data_for_qualtrics.csv")
# Reduce story_df to columns of interest
story_df = story_df[["Story_ID", "full_story", "prompt", "memory", "result"]]
story_df.rename(columns={"Story_ID": "story_id"}, inplace=True)

## Store Question Texts For Items

In [2]:
# The first row has in the response dfs has descriptions of the variables
# This also includes the wording of the items, which will be useful for later
# I will save these to its own df
items_descr = panel_df.iloc[0, 18:94]

print(f"Question text for first 3 items:\n{items_descr.head(3)}")

# I got everything I need from the first row of the main dfs - delete it
panel_df.drop(index=[0, 1], inplace=True)
community_df.drop(index=[0, 1], inplace=True)

Question text for first 3 items:
Qual_Check_1                    The story I just read, began with...
Qual_Check_1_DO    The story I just read, began with... - Display...
story_scale_1      For the following questions, please think of t...
Name: 0, dtype: object


In [3]:
# Quick clean up of the question descriptions
q_intro_to_remove = """For the following questions, please think of the story you just read.
Indicate how much you agree or disagree with each of the following statements about the story. - """

items_descr = items_descr.str.replace(q_intro_to_remove, "", regex=False)

print(f"Cleaned question text for first 3 items:\n\n{items_descr.head(3)}")

Cleaned question text for first 3 items:

Qual_Check_1                    The story I just read, began with...
Qual_Check_1_DO    The story I just read, began with... - Display...
story_scale_1      I had a hard time making sense of what was goi...
Name: 0, dtype: object


## Delete Cases Without Any Response on Question Items

In [4]:
# Get list of story items column names
story_items_cols = community_df.iloc[:,18:91].columns

# Drop cases without any response on the story items
community_df.dropna(how="all", inplace=True, subset=story_items_cols)
panel_df.dropna(how="all", inplace=True, subset=story_items_cols)

# Get count of cases with 1+ response on the story items
print("Initial case count")
print("Community sample: {}\nPanel sample: {}".format(community_df.shape[0],panel_df.shape[0]))

Initial case count
Community sample: 165
Panel sample: 233


## Various Data Wrangling Stuff
Here I am running a bunch of data wrangling procedures on both response dataframes, before I combine them (mostly data cleaning and adding new variables).

In [5]:
response_dfs = [community_df, panel_df]

for i, df in enumerate(response_dfs):
    # Delete identifying or unnecessary columns:
    cols_to_delete = ['Status', 'IPAddress', 'Progress', 'Finished', 'RecipientLastName', 'RecipientFirstName',
                  'RecipientEmail', 'ExternalReference', 'LocationLatitude', 'LocationLongitude',
                  'DistributionChannel', 'UserLanguage', '1']
    df.drop(columns=cols_to_delete, inplace=True)

    # story_id:
    # Qualtrics saves the order in which randomized items were shown in seperate variables
    # For the story question block, only one "item" was shown: The story the participant rated
    # --> The display order variable for this block will contain only one item:
    #     the Question ID of of the story item
    # --> We can treat the display order variable for the story block as story ID variable
    # --> rename accordingly
    df.rename(
        columns={"Stories-Feb17,2022_DO": "story_id"}, inplace=True)
    
    # A few stories were repeated to make the design balanced
    # Those have a "_rep" in their name
    # cut the "_rep" from these names so that identical stories have identical IDs 
    df["story_id"] = df["story_id"].str.replace(r"_rep", "")
    
    # prompt_label:
    # Story IDs contain abbreviations for the preset & prompt
    # Extract and save prompt abbreviation from story_id
    df["prompt_label"] = df["story_id"].str.extract(r"_(.*)_\d_?\d?")
    # Rename prompt abbreviation to full name
    prompt_rename_dict = {"HF": "High Fantasy", "HOR": "Horror",
                        "HR": "Historical Romance", "HSF": "Hard Sci-Fi"}
    df["prompt_label"].replace(prompt_rename_dict, inplace=True)

    # preset_label:
    # Extract preset abbreviation
    df["preset_label"] = df["story_id"].str.extract(r"^([A-Z]*)")
    # Rename presett abbreviation to full name
    preset_rename_dict = {"ACE": "Ace of Spade", "ALL": "All-Nighter",
                        "BAS": "Basic Coherence", "FAN": "Fandango",
                        "GEN": "Genesis", "LOW": "Low Rider",
                        "MOR": "Morpho", "OUR": "Ouroboros"}
    df["preset_label"].replace(preset_rename_dict, inplace=True)

    # Add story infos
    df = df.join(story_df.set_index("story_id"), on="story_id", how="left")

    # Recode likert responses:
    likert_recode_dict = {'Strongly disagree': 1, 'Somewhat disagree': 2,
                      'Neither agree nor disagree': 3,
                      'Somewhat agree': 4, 'Strongly agree': 5}
    df.replace(likert_recode_dict, inplace=True)

    # Add word count per story:
    df["word_count"] = df["full_story"].str.split().apply(len)

    # Add readability scores:
    df["read_consesus"] = df["full_story"].apply(lambda x: txt.text_standard(x, float_output=True))
    df["read_fre"] = df["full_story"].apply(lambda x: txt.flesch_reading_ease(x))

    response_dfs[i] = df

community_df = response_dfs[0]
panel_df = response_dfs[1]

## Data Quality Check For Panel Data
The panel survey contained 2 quality checks:
1. Directly after the participant read the story excerpt, she was asked to identify the beginning of the story she just read
2. One quality check question to check general attention was mixed into the story scale ("This is a quality check. Please select "Somewhat disagree".")

Below, I will sort out participants that did not pass these checks. I will treat all participants failing 2) as bad respondents either way. Failing only quality check 1) will lead to a manual check.

In [6]:
# Label if participants passed quality checks
def evaluate_qual_check_1(row):
    # returns True if passed check
    if (row["Qual_Check_1"] == '...someone working in a tavern.') and (row["prompt_label"] == "Historical Romance"):
        return True
    elif (row["Qual_Check_1"] == '...a noise coming from a mirror.') and (row["prompt_label"] == "Horror"):
        return True
    elif (row["Qual_Check_1"] == '...with a message from the president.') and (row["prompt_label"] == "Hard Sci-Fi"):
        return True
    elif (row["Qual_Check_1"] == '...the description of a small village.') and (row["prompt_label"] == "High Fantasy"):
        return True
    else:
        return False

# Evaluate quality check 1 - participants correctly identified beginning of story
panel_df["pass_qual_1"] = panel_df.apply(
    evaluate_qual_check_1, axis=1)
# Check quality check 2 - participants were asked to give specific answer to question
panel_df["pass_qual_2"] = panel_df.apply(
    lambda row: True if row["story_scale_74"] == 2 else False, axis=1)

# Failing quality check 2 leads to  bad respondent label either way
# If quality check 1 is failed, but 2 is passed needs closer look
mask_inspect_qual = (panel_df["pass_qual_1"] == False) & (
    panel_df["pass_qual_2"] == True)
inspect_count = mask_inspect_qual.value_counts()[True]
print("{} cases need manual investigation. Run cell below to review them.".format(inspect_count))

14 cases need manual investigation. Run cell below to review them.


In [7]:
# Create final check column - failed either way if quality check 2 was failed
# Otherwise will go through manual check
panel_df["passed_manual_check"] = panel_df["pass_qual_2"]

for index,row in panel_df[mask_inspect_qual].iterrows():
    print("Participant said this story began with...")
    print(row["Qual_Check_1"], "\n")
    print("Actual story was:")
    print(row["full_story"], "\n\n")
    manual_qual_check = input("The participant was correct (yes/no): ")
    panel_df.at[index,'passed_manual_check'] = True if (manual_qual_check == "yes") else False

Participant said this story began with...
...the description of a small village. 

Actual story was:
"I have a message for you from the president," said Dr. Sato, handing over an envelope to me. "He's asking that we meet with him at his office this afternoon." I took it and thanked her before walking out of my apartment building into the bright sun. It was already noon on Mars—the longest day in the year here on the planet. The air felt warm against my face as I walked down the street toward the presidential palace.
The streets were empty except for a few people going about their daily business. I saw one old woman pushing a cart filled with vegetables, another man carrying two large bags of grain across his shoulders, and a young couple holding hands while they walked past me. As I passed them, I could see the girl looking up at me curiously. She had dark brown skin like mine but wore a white dress with purple trim. Her hair was braided tightly around her head, and she looked familiar

In [8]:
# create dataframe with only good respondents
mask_passed = (panel_df["passed_manual_check"] == True)
panel_df = panel_df[mask_passed].copy(deep=True)

print("\nCase count of panel sample after deleting failed quality checks for panel data: {}".format(panel_df.shape[0]))


Case count of panel sample after deleting failed quality checks for panel data: 161


## Combine Response Dataframes
After sorting out bad respondents from the panel data, it is time to combine the community and panel sample.

In [9]:
# Time to combine community and SurveySwap sample
community_df["sample"] = "Community"
panel_df["sample"] = "Panel"
combined_df = pd.concat([community_df, panel_df])

In [10]:
# Rename remaining columns to a more sensible and easier to use naming-scheme

# build mapping dict for renaming
rename_cols_dict_1 = {'StartDate': "start", 'EndDate': "end", 'Duration (in seconds)': "duration_in_sec",
                      'RecordedDate': "recorded", 'ResponseId': "response_id",
                      "story_scale_DO": "ss_order",
                      "Qual_Check_1": "qual_check_1", "Qual_Check_1_DO": "qual_check_1_order"}

rename_cols_dict_2 = {}

for i in range(1, 13):
    current_item = "story_scale_" + str(i)
    new_item = "ss_coh_" + str(i)
    rename_cols_dict_2[current_item] = new_item

a = 1
for i in range(13, 21):
    current_item = "story_scale_" + str(i)
    new_item = "ss_conch_" + str(a)
    rename_cols_dict_2[current_item] = new_item
    a += 1

a = 1
for i in range(21, 33):
    current_item = "story_scale_" + str(i)
    new_item = "ss_cre_" + str(a)
    rename_cols_dict_2[current_item] = new_item
    a += 1

a = 1
for i in range(33, 42):
    current_item = "story_scale_" + str(i)
    new_item = "ss_qua_" + str(a)
    rename_cols_dict_2[current_item] = new_item
    a += 1

a = 1
for i in range(42, 53):
    current_item = "story_scale_" + str(i)
    new_item = "ss_rep_" + str(a)
    rename_cols_dict_2[current_item] = new_item
    a += 1

a = 1
for i in range(53, 65):
    current_item = "story_scale_" + str(i)
    new_item = "ss_sty_" + str(a)
    rename_cols_dict_2[current_item] = new_item
    a += 1

a = 1
for i in range(65, 74):
    current_item = "story_scale_" + str(i)
    new_item = "ss_pac_" + str(a)
    rename_cols_dict_2[current_item] = new_item
    a += 1

rename_cols_dict_2["story_scale_74"] = "qual_check_2"

# update items_descr naming
items_descr.rename(rename_cols_dict_2, inplace=True)

# update naming for combined_df
combined_df.rename(columns=rename_cols_dict_1, inplace=True)
combined_df.rename(columns=rename_cols_dict_2, inplace=True)

In [12]:
combined_df.columns

Index(['start', 'end', 'duration_in_sec', 'recorded', 'response_id',
       'ss_coh_1', 'ss_coh_2', 'ss_coh_3', 'ss_coh_4', 'ss_coh_5', 'ss_coh_6',
       'ss_coh_7', 'ss_coh_8', 'ss_coh_9', 'ss_coh_10', 'ss_coh_11',
       'ss_coh_12', 'ss_conch_1', 'ss_conch_2', 'ss_conch_3', 'ss_conch_4',
       'ss_conch_5', 'ss_conch_6', 'ss_conch_7', 'ss_conch_8', 'ss_cre_1',
       'ss_cre_2', 'ss_cre_3', 'ss_cre_4', 'ss_cre_5', 'ss_cre_6', 'ss_cre_7',
       'ss_cre_8', 'ss_cre_9', 'ss_cre_10', 'ss_cre_11', 'ss_cre_12',
       'ss_qua_1', 'ss_qua_2', 'ss_qua_3', 'ss_qua_4', 'ss_qua_5', 'ss_qua_6',
       'ss_qua_7', 'ss_qua_8', 'ss_qua_9', 'ss_rep_1', 'ss_rep_2', 'ss_rep_3',
       'ss_rep_4', 'ss_rep_5', 'ss_rep_6', 'ss_rep_7', 'ss_rep_8', 'ss_rep_9',
       'ss_rep_10', 'ss_rep_11', 'ss_sty_1', 'ss_sty_2', 'ss_sty_3',
       'ss_sty_4', 'ss_sty_5', 'ss_sty_6', 'ss_sty_7', 'ss_sty_8', 'ss_sty_9',
       'ss_sty_10', 'ss_sty_11', 'ss_sty_12', 'ss_pac_1', 'ss_pac_2',
       'ss_pac_3', 'ss_pac_4

In [13]:
# reorder cols
cols_new_order = ['response_id', 'story_id', 'prompt_label', 'preset_label', 'sample',
                  'ss_coh_1', 'ss_coh_2', 'ss_coh_3', 'ss_coh_4', 'ss_coh_5',
                  'ss_coh_6', 'ss_coh_7', 'ss_coh_8', 'ss_coh_9', 'ss_coh_10',
                  'ss_coh_11', 'ss_coh_12', 'ss_conch_1', 'ss_conch_2', 'ss_conch_3',
                  'ss_conch_4', 'ss_conch_5', 'ss_conch_6', 'ss_conch_7',
                  'ss_conch_8', 'ss_cre_1', 'ss_cre_2', 'ss_cre_3', 'ss_cre_4',
                  'ss_cre_5', 'ss_cre_6', 'ss_cre_7', 'ss_cre_8', 'ss_cre_9',
                  'ss_cre_10', 'ss_cre_11', 'ss_cre_12', 'ss_qua_1', 'ss_qua_2',
                  'ss_qua_3', 'ss_qua_4', 'ss_qua_5', 'ss_qua_6', 'ss_qua_7',
                  'ss_qua_8', 'ss_qua_9', 'ss_rep_1', 'ss_rep_2', 'ss_rep_3', 'ss_rep_4',
                  'ss_rep_5', 'ss_rep_6', 'ss_rep_7', 'ss_rep_8', 'ss_rep_9',
                  'ss_rep_10', 'ss_rep_11', 'ss_sty_1', 'ss_sty_2',
                  'ss_sty_3', 'ss_sty_4', 'ss_sty_5', 'ss_sty_6', 'ss_sty_7',
                  'ss_sty_8', 'ss_sty_9', 'ss_sty_10', 'ss_sty_11', 'ss_sty_12',
                  'ss_pac_1', 'ss_pac_2', 'ss_pac_3', 'ss_pac_4', 'ss_pac_5',
                  'ss_pac_6', 'ss_pac_7', 'ss_pac_8', 'ss_pac_9',
                  'full_story', 'word_count', 'prompt', 'memory', 'result',
                  'start', 'end', 'duration_in_sec', 'recorded',
                  'ss_order',
                  'qual_check_1', 'qual_check_1_order', 'qual_check_2', 'pass_qual_1', 'pass_qual_2']
combined_df = combined_df[cols_new_order]

## Archived: Filter Out Extreme Response Durations

The following code was an attempt to improve data quality by sorting out extreme response durations. This did not work too well and seems to sort out valid responses. 

The reason might be that the method sorted out responses that opened the start of the survey, immediately closed it and then reopened and finished at a later time. The result would be a totally valid response, but it would be recorded as an extreme response duration.

Importantly, the method also failed to detect speeders

The code is just archived/commented out here for now...

In [ ]:
# # Sort out extreme long or short response times

# combined_df["duration_in_sec"] = pd.to_numeric(
#     combined_df["duration_in_sec"])

# %matplotlib inline

# fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# # Histograms with KDE
# ax1 = combined_df["duration_in_sec"].plot.hist(ax=axes[0], bins=15)
# ax1.set_xlabel("duration_in_sec")
# combined_df["duration_in_sec"].plot.kde(ax=axes[0], secondary_y=True)

# # Boxplots
# combined_df["duration_in_sec"].plot.box(ax=axes[1])

# fig.tight_layout()
# plt.show()

In [ ]:
# # Use Turkey's rule for exreme outliers to sort out extreme response times (no more than IQRx3 from Q1 or Q3)
# def detect_outliers(df, check_col_label):
#     q1 = df[check_col_label].quantile(0.25)
#     q3 = df[check_col_label].quantile(0.75)
#     iqr = q3-q1
#     outlier_lower = q1 - (iqr*1.5)
#     outlier_upper = q3 + (iqr*3)
#     print("25th Percentile (Q1): {}\n75th Percentile (Q3): {}\nIQR: {}".format(q1, q3, iqr))
#     print("will detect outliers with values lower than {} or higher than {}".format(
#         outlier_lower, outlier_upper))

#     out_series = df[check_col_label].apply(lambda x: True if ((x < outlier_lower) or (x > outlier_upper)) else False)
#     count_out = df[out_series == True].shape[0]
    
#     print("\n{} Outliers".format(count_out))
#     if count_out > 0:
#         print(df[check_col_label][out_series == True])
    
#     return(out_series, count_out)

In [ ]:
# # Sort out outliers until check comes up with no extreme outliers

# combined_df["Duration (in seconds)"] = pd.to_numeric(combined_df["Duration (in seconds)"])
# count_outliers = 99

# while count_outliers > 0:
#     out_result = detect_outliers(combined_df, "Duration (in seconds)")
#     count_outliers = out_result[1]
#     out_series = out_result[0]

#     mask_good_resp = out_series != True
#     combined_df = combined_df[mask_good_resp]

In [ ]:
# # Plot new duration distribution
# %matplotlib inline

# fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# # Histograms with KDE
# ax1 = combined_df["duration_in_sec"].plot.hist(ax=axes[0], bins=15)
# ax1.set_xlabel("duration_in_sec")
# combined_df["duration_in_sec"].plot.kde(ax=axes[0], secondary_y=True)

# # Boxplots
# combined_df["duration_in_sec"].plot.box(ax=axes[1])

# fig.tight_layout()
# plt.show()

## Save Cleaned Data To File

In [14]:
print("Case count after cleaning:")
print("Community sample: {}\nPanel Sample: {}\nTotal Sample: {}".format(
    combined_df[combined_df["sample"] == "Community"].shape[0],
    combined_df[combined_df["sample"] == "Panel"].shape[0],
    combined_df.shape[0]))

Case count after cleaning:
Community sample: 165
Panel Sample: 161
Total Sample: 326


In [15]:
# save to file
combined_df.to_csv("data/combined_data.csv")
items_descr.to_csv("data/description_items.csv")

Files are now prepped for scale construction/exploratory factor analysis (EFA).

See you in [part 2](step_2_scale_construction_efa.ipynb) for the EFA!